In [1]:
import numpy as np
from scipy import stats
from scipy.stats import chi2_contingency,ttest_ind,mannwhitneyu,kstest
from sklearn.metrics import confusion_matrix
import pandas as pd
import json

In [2]:
import pandas as pd 
data = pd.read_excel('./data/清洗数据-0714.xlsx')

In [5]:
# 定量分析的清单:数值类型的数据
quantitative_analysis = ["心率","舒张压","收缩压","年龄", "LDL-C检测值", "HDL-C检测值",
                         "TC检测值", "空腹血糖检测值","最高值肌酐检测值","首次血红蛋白检测值",
                        "CK-MB最高值检测值","TG检测值", "肌钙蛋白I最高值检测值","肌钙蛋白T最高值检测值",
                        "肌红蛋白最高值检测值","糖化血红蛋白检测值","BNP检测值","NT-proBNP检测值","LVEF","乳酸脱氢酶最高值检测值.2",
                        "TyG指数", "TG/HDL"]



# 定性分析的清单: 标签类型的数据 男女 是否
qualitative_analysis = ["性别","戒烟：无","吸烟史","饮酒史","既往心梗","脑血管病史",
                        '合并高血压','合并糖尿病',"既往心梗","心衰史","既往CABG","既往PCI",
                        "本次住院是否进行CABG治疗","室壁瘤.1","是否行PCI",
                        "乳头肌功能不全或断裂","室壁破裂","室间隔穿孔","附壁血栓","室壁瘤","并发症其他",
                        "冠状动脉支架植入术后再狭窄","冠状动脉支架内血栓形成","急性亚急性支架血栓形成","急性再发心肌梗死","再梗死","消化道出血.1","心脏瓣膜病",
                        "其他出血事件",
                        "入院诊断","首次心功能分级","出院主要诊断","LM_LAD_LCX_RCA" # 不是二值变量
                       ] 

In [6]:
save_list = []

def generate_array(demo_df, col_name):
    group_stat = demo_df.value_counts([col_name,'mace']).reset_index().rename({0:"count"},axis=1)
    group_df = group_stat.pivot(index=col_name,columns='mace',values='count')
    # print(group_df)
    return np.array(group_df)

for col_name in qualitative_analysis: # 对定性分析的数据进行 卡方检验
    # 输出内容： mace 组的值 非mace组的各值
    # print(col_name)
    if len(data[col_name].dropna()) < 1134:
        # print(f"{col_name}")
        print(data[col_name].value_counts())
    cur_mace_info = mace_df[col_name].value_counts().to_dict()
    cur_non_mace_info = non_mace_df[col_name].value_counts().to_dict()
    
    # print(data[col_name].value_counts())
    cal_mat = generate_array(data,col_name)
    chi2_res = chi2_contingency(cal_mat)
    # print()
    save_list.append([col_name,
                      json.dumps(cur_mace_info,ensure_ascii=False), 
                     json.dumps(cur_non_mace_info,ensure_ascii=False),
                     round(chi2_res.pvalue,4)])
save_df = pd.DataFrame(save_list,columns=["指标","mace","非mace","p-value"])

是否行PCI
是     1049
否       60
不详       2
Name: count, dtype: int64


In [7]:
mace_total = 46 + 13
non_mace_total = 310 + 91
mace_ratio, non_mace_ratio = [], []
for idx, col in save_df.iterrows():
    mace_cont = json.loads(col['mace'])
    non_mace_cont = json.loads(col['非mace'])
    if "是" in mace_cont:
        mace_cnt = mace_cont['是']
        non_mace_cnt = non_mace_cont.get('是',0)
    elif "true" in mace_cont:
        mace_cnt = mace_cont['true']
        non_mace_cnt = non_mace_cont['true']
    else:
        label = list(mace_cont.keys())[0]
        mace_cnt  = mace_cont[label]
        non_mace_cnt  = non_mace_cont[label]
    mace_ratio.append(f"{round(mace_cnt * 100 / mace_total, 2)}%")
    non_mace_ratio.append(f"{round(non_mace_cnt * 100 / non_mace_total, 2)}%")   
save_df['mace比例'] = mace_ratio
save_df['测试集比例'] = non_mace_ratio
dx_df = save_df # 定性分析结果
del save_df

In [8]:
dx_df.to_excel('./data/定性分析结果-0826.xlsx',index=None)

In [9]:
save_list = []
for col_name in quantitative_analysis: # 对定量分析的数据进行 先判断是否符合正正态分布 满足t检验 不满足 u检验
    # 输出内容： mace 组的值 非mace组的各值
    data_mean,data_std = np.mean(data[col_name]), np.std(data[col_name]) 
    norm_res = stats.kstest(data[col_name], 'norm', (data_mean, data_std))
    if norm_res.pvalue > 0.05: # 大于0.05就是正态分布
        is_norm = True
    else:
        is_norm = False
    
    # print(col_name)
    # 填充空值
    mace_df[col_name] = mace_df[col_name].fillna(value=mace_df[col_name].mean())
    non_mace_df[col_name] = non_mace_df[col_name].fillna(value=non_mace_df[col_name].mean())
    
    
    # 原有执行流程
    cur_mace_arr = mace_df[col_name].dropna()
    mace_mean, mace_std = np.mean(cur_mace_arr), np.std(cur_mace_arr)
    cur_non_mace_arr = non_mace_df[col_name].dropna()
    # if len(cur_mace_arr) + len(cur_non_mace_arr) != 1134:
    #     print(f"{col_name}\t {len(cur_mace_arr) + len(cur_non_mace_arr)}")
    no_mace_mean, no_mace_std = np.mean(cur_non_mace_arr), np.std(cur_non_mace_arr)
    
    # 对数据进行是否正态分布的验证
    statistic_t, p_value_t = ttest_ind(cur_mace_arr, cur_non_mace_arr)

    statistic_u, p_value_u = mannwhitneyu(cur_mace_arr,cur_non_mace_arr,alternative='two-sided')
    mace_quater_arr1, mace_quater_arr2,mace_quater_arr3 = np.percentile(cur_mace_arr, [25, 50, 75]).tolist()
    non_quater_arr1, non_quater_arr2,non_quater_arr3 = np.percentile(cur_non_mace_arr, [25, 50, 75]).tolist()
    
    # print(data[col_name].value_counts())
    # cal_mat = generate_array(data,col_name)
    # chi2_res = chi2_contingency(cal_mat)
    # print()
    save_list.append([col_name,
        f"{round(mace_mean,2)}±{round(mace_std,2)}",
        f"{round(no_mace_mean,2)}±{round(no_mace_std,2)}",
          round(statistic_t,4),
          f"{round(mace_quater_arr2,2)}({round(mace_quater_arr1,2)},{round(mace_quater_arr3,2)})",
          f"{round(non_quater_arr2,2)}({round(non_quater_arr1,2)},{round(non_quater_arr3,2)})",
         round(p_value_t,4),
         round(statistic_u,4),
         round(p_value_u,4),
         is_norm ])
save_df = pd.DataFrame(save_list,columns=["指标","mace","非mace","t统计值","mace四分位数","非mace四分位数", "p-value-t", "u统计值", "p-value-u", "是否正态分布"])

/tmp/ipykernel_90794/1646894356.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  mace_df[col_name] = mace_df[col_name].fillna(value=mace_df[col_name].mean())
/tmp/ipykernel_90794/1646894356.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  non_mace_df[col_name] = non_mace_df[col_name].fillna(value=non_mace_df[col_name].mean())


In [10]:
dl_df = save_df # 定量分析结果
del save_df
dl_df.to_excel('./data/定量分析结果-0826.xlsx',index=None)